## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

#import keys by other way
import sys 
sys.path.append("C:/Users/Ameysne/api")
from api_key import g_key
api_key = g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_DataBase/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zhigansk,RU,66.7697,123.3711,-48.87,99,76,3.36,broken clouds
1,1,Sucua,EC,-2.4667,-78.1667,86.74,45,20,5.75,light rain
2,2,Ushuaia,AR,-54.8000,-68.3000,58.66,47,75,14.97,broken clouds
3,3,Castro,BR,-24.7911,-50.0119,72.66,59,85,7.67,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,78.35,87,88,12.93,overcast clouds


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Castro,BR,-24.7911,-50.0119,72.66,59,85,7.67,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,78.35,87,88,12.93,overcast clouds
7,7,Carnarvon,AU,-24.8667,113.6333,71.67,83,75,18.41,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,75.18,76,62,3.00,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,77.16,77,96,9.13,overcast clouds
13,13,Pemba,MZ,-12.9740,40.5178,79.50,88,20,7.00,few clouds
15,15,Atuona,PF,-9.8000,-139.0333,78.73,78,24,21.25,few clouds
17,17,Yeppoon,AU,-23.1333,150.7333,82.18,64,34,5.19,scattered clouds
20,20,Faanui,PF,-16.4833,-151.7500,82.29,68,70,15.32,broken clouds
23,23,Jimeta,NG,9.2833,12.4667,77.18,24,2,2.71,clear sky


In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                185
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Castro,BR,-24.7911,-50.0119,72.66,59,85,7.67,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,78.35,87,88,12.93,overcast clouds
7,7,Carnarvon,AU,-24.8667,113.6333,71.67,83,75,18.41,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,75.18,76,62,3.00,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,77.16,77,96,9.13,overcast clouds


In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Castro,BR,72.66,overcast clouds,-24.7911,-50.0119,
4,Saint-Philippe,RE,78.35,overcast clouds,-21.3585,55.7679,
7,Carnarvon,AU,71.67,broken clouds,-24.8667,113.6333,
10,Kapaa,US,75.18,light rain,22.0752,-159.3190,
11,Rikitea,PF,77.16,overcast clouds,-23.1203,-134.9692,
13,Pemba,MZ,79.50,few clouds,-12.9740,40.5178,
15,Atuona,PF,78.73,few clouds,-9.8000,-139.0333,
17,Yeppoon,AU,82.18,scattered clouds,-23.1333,150.7333,
20,Faanui,PF,82.29,broken clouds,-16.4833,-151.7500,
23,Jimeta,NG,77.18,clear sky,9.2833,12.4667,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
            
            
hotel_df.head(10)            
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Castro,BR,72.66,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
4,Saint-Philippe,RE,78.35,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Carnarvon,AU,71.67,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Kapaa,US,75.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Rikitea,PF,77.16,overcast clouds,-23.1203,-134.9692,People ThankYou
13,Pemba,MZ,79.50,few clouds,-12.9740,40.5178,Guesthouse Samotina
15,Atuona,PF,78.73,few clouds,-9.8000,-139.0333,Villa Enata
17,Yeppoon,AU,82.18,scattered clouds,-23.1333,150.7333,Villa Mar Colina
20,Faanui,PF,82.29,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
23,Jimeta,NG,77.18,clear sky,9.2833,12.4667,Yola International Hotel


In [57]:
import numpy as np
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Castro,BR,72.66,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
4,Saint-Philippe,RE,78.35,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Carnarvon,AU,71.67,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Kapaa,US,75.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Rikitea,PF,77.16,overcast clouds,-23.1203,-134.9692,People ThankYou


In [58]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))